In [1]:
import torch
from torch.autograd import Variable
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from net import CapsNetWithReconstruction, CapsNet, ReconstructionNet, Mask
import numpy as np
from scipy import stats
from torchvision.utils import make_grid, save_image

def get_digit_caps(model, image):
    input_ = Variable(image.unsqueeze(0), volatile=True)
    digit_caps, probs = model.capsnet(input_)
    return digit_caps

def get_mask(model, digit_caps, label):
    target = Variable(torch.LongTensor([label]), volatile=True)
    digit_caps_mask = model.mask(digit_caps, target)
    return digit_caps_mask

capsnet = CapsNet(routing_iterations=3, n_classes=10, capsuleblock=16, n_dim=8)
mask = Mask(10)
reconstructionnet = ReconstructionNet(n_dim=8, n_classes=10)
model = CapsNetWithReconstruction(capsnet, mask, reconstructionnet)

MODEL_PATH = r'trained_model\MNIST_99epoch_dim12_kl0.5_rec0.0117_kl0.0028.pth'
model.load_state_dict(torch.load(MODEL_PATH))

RuntimeError: Error(s) in loading state_dict for CapsNetWithReconstruction:
	size mismatch for capsnet.digitCaps.weights: copying a param with shape torch.Size([576, 8, 12]) from checkpoint, the shape in current model is torch.Size([576, 8, 80]).
	size mismatch for capsnet.digitCaps.routing_module.b: copying a param with shape torch.Size([576, 1]) from checkpoint, the shape in current model is torch.Size([576, 10]).
	size mismatch for reconstruction_net.fc1.weight: copying a param with shape torch.Size([512, 12]) from checkpoint, the shape in current model is torch.Size([512, 8]).

In [ ]:
test_data = datasets.MNIST('../data', train=False,
                            transform=ToTensor())
i = 0
image_tensor, label = test_data[i]
digit_caps = get_digit_caps(model, image_tensor)
digit_caps_mask = get_mask(model, digit_caps, label)
std = digit_caps_mask

for i in range(1,1000):
    image_tensor, label = test_data[i]
    digit_caps = get_digit_caps(model, image_tensor)
    digit_caps_mask = get_mask(model, digit_caps, label)
    std = torch.cat([std, digit_caps_mask], 0)

In [ ]:
std.std(dim=0)

In [ ]:
std.std(dim=0).mean()